# Install requerements

In [5]:
import requests
from tabula import read_pdf
import pandas as pd
from pathlib import Path

# Download the PDF

In [6]:
pdf_path = Path("/Users/marco/Library/CloudStorage/OneDrive-UniversitàdegliStudidiMilano/PersDocs/Banking-Financial/RendicontazioneAnnuale/2024/TradeRepublic/TR-Lug-Sept2024.pdf")
pdf_folder = pdf_path.parent

view images

In [ ]:
#!pip install pdf2image
#!apt-get install poppler-utils

In [ ]:
from pdf2image import convert_from_path
import numpy as np
from matplotlib import pyplot as plt

images_of_pdf = convert_from_path(pdf_path, dpi=200)  # Convert PDF to List of PIL Images
imgs_path = []

for i,page in enumerate(images_of_pdf):
    page.save(str(pdf_folder) + "/page_image_"+str(i)+".jpg")
    imgs_path.append(str(pdf_folder) + "/page_image_"+str(i)+".jpg")

readable_images_of_pdf = []  # Create a list for thr for loop to put the images into
for PIL_Image in images_of_pdf:
    readable_images_of_pdf.append(np.array(PIL_Image))  # Add items to list

n_pages = len(images_of_pdf)
print("n pages exported: ", n_pages)
type(readable_images_of_pdf)

let's load the images to have a preview of the modifications we will do for tabula

In [10]:
#from google.colab.patches import cv2_imshow
import cv2

#reload the img in cv2
imgs= []
for i,page in enumerate(imgs_path):
    imgs.append(cv2.imread(page, cv2.IMREAD_UNCHANGED))


In [ ]:
# Display the image
cv2.imshow('Image', imgs[0])
# Wait for 1 millisecond
cv2.waitKey(1)

In [ ]:
# Close the window
cv2.destroyWindow('Image')
cv2.waitKey(1) #this will ensure the closure of the event loop and prevent the window to reopen

In [ ]:
#resize the img to the original values of the pdf. To get them open it with any pdf reader and get those info or look in the file properties. Values are in pts.
for i, img in enumerate(imgs):
    imgs[i] = cv2.resize(img, (int(595.28), int(841.88)), interpolation=cv2.INTER_LINEAR)

# Get the dimensions of the image
height, width, channels = img.shape

print(f"Width: {width}, Height: {height}, Channels: {channels}")

In [59]:
# Define the x points for vertical lines
x_points = [80, 148, 443, 475, 507, 555]  # Example x points

# Define the area to color (top, left, bottom, right)
area = [59.9,42.042,790.606,556.959]
area_p1 = [262.295,42.042,789.118,552.494]  #the area for the first page
area_last = [59.9,42.042,790.606,556.959] #the area for the last page where u have a final piece of text

In [ ]:
#MODIFY THIS
area_preview = area #area_p1 or area
img = imgs[2].copy()

#reset the image every time
# img = cv2.imread("/content/page_image_0.jpg", cv2.IMREAD_UNCHANGED)
# img = cv2.resize(img, (int(595.28), int(841.88)), interpolation=cv2.INTER_LINEAR)

overlay = np.ones((img.shape[0], img.shape[1], 4), dtype='uint8')

# Convert the original image to BGRA (with alpha channel)
img_bgra = cv2.cvtColor(img, cv2.COLOR_BGR2BGRA)

# Color the defined area
cv2.rectangle(overlay, (int(area_preview[1]), int(area_preview[0])), (int(area_preview[3]), int(area_preview[2])), (255, 0, 0, 125), -1)  # Blue filled rectangle

# Draw vertical lines
for x in x_points:
    cv2.line(overlay, (x, 0), (x, img.shape[0]), (0, 255, 0,255), 2)  # Green vertical lines

# Blend the overlay with the original image
combined = cv2.addWeighted(img_bgra, 0.9, overlay, 0.2, 0)

# Display the image
cv2.imshow('overlay', combined)
cv2.waitKey(1)

In [ ]:
# Close the window
cv2.destroyWindow('overlay')
cv2.waitKey(1) #this will ensure the closure of the event loop and prevent the window to reopen

# Extract tables

In [62]:
pandas_options = {
    'header': None,
    'columns' : ["DATA", "TIPO", "DESCRIZIONE", "IN ENTRATA","IN USCITA", "SALDO"]
}

Inspect the pdf, if in the last page u don't have any data, execute following cell and change the value of **extract_last_page**

In [63]:
extract_last_page = False
if not extract_last_page:
    n_pages = n_pages-1

In [ ]:
with open(pdf_path, 'rb') as file:
        # extract table from pages
        dfs = read_pdf(
            file,
            pages="2-" + str(n_pages), # pages to extract table from
            area=area,
            columns=x_points,
            #lattice=True,
            pandas_options=pandas_options,
            stream=True,
        )

        # extract table in page 1
        dfs_1 = read_pdf(
            file,
            pages=1,
            area=area_p1,
            columns=x_points,
            pandas_options=pandas_options,
            stream = True,
        )

        if extract_last_page:
            #extract table in last page
            dfs_2 = read_pdf(
                file,
                pages=n_pages,
                area=area_p1,
                columns=x_points,
                pandas_options=pandas_options,
                stream = True,
            )
dfs[0]
#dfs_1[0]
#dfs_2[0]

In [ ]:
dfs.extend(dfs_1)
if extract_last_page:
    dfs.extend(dfs_2)

# concat all dfs to get one table
data = pd.concat(
    dfs,
    axis=0,
    ignore_index=True
)
data

In [ ]:
# Drop rows if they contain all column names
try:
    data = data.drop(data[data.apply(lambda row: all(item in row.values for item in pandas_options['columns']), axis=1)].index)
    # Reset the index
    data = data.reset_index(drop=True)
except KeyError:
    ("everything ok")
data


In [ ]:
# Group the DataFrame by every 3 rows as tabula didn't recognize multiline text in the cells. U should inspect this by urself
grouped = data.groupby(data.index // 3)

# Aggregate the text within each group
result = grouped.agg(lambda x: ''.join(x.fillna('')))

# Reset the index
result = result.reset_index(drop=True)

result

# Export data to CSV

In [69]:
result.to_csv(str(pdf_folder) + "/" + pdf_path.stem + "output.csv", index=False)